In [33]:
import re

from datasets import load_dataset
import pandas as pd

In [34]:
df = pd.read_csv('data/annotated_fb_data_valid.txt', delimiter='\t', header=None)

In [35]:
df.columns = ['useless3', 'useless1', 'useless2', 'question']
df.drop(columns=['useless1', 'useless2', 'useless3'], inplace=True)
df

,question
0,Who was the trump ocean club international hot...
1,where was sasha vujačić born
2,What is a region that dead combo was released in
3,What is a film directed by wiebke von carolsfeld?
4,what country was music for stock exchange rel...
...,...
10840,What is nikolaj frobenius's profession?
10841,who is a person that was born in sao paulo
10842,What area does zakynthos international airport...
10843,what kind of personality do german wirehaired ...


In [36]:
# Remove punctuation for all questions: Meeting transcripts won't have it
import re
def remove_punc(a_str):
    return re.sub(r'[^\w\s]', '', a_str).lower()

In [37]:
df['question'] = df['question'].apply(remove_punc)

In [38]:
df

,question
0,who was the trump ocean club international hot...
1,where was sasha vujačić born
2,what is a region that dead combo was released in
3,what is a film directed by wiebke von carolsfeld
4,what country was music for stock exchange rel...
...,...
10840,what is nikolaj frobeniuss profession
10841,who is a person that was born in sao paulo
10842,what area does zakynthos international airport...
10843,what kind of personality do german wirehaired ...


In [39]:
# Split dataframes to be nice to openAI
num_splits = (len(df) + 500 - 1) // 500 
subframes = [df.iloc[i*500 : (i+1)*500] for i in range(num_splits)]
# trim to dataset of 4000- Limit due to API Call
subframes= subframes[0:8]
len(subframes)

8

In [40]:
# Open AI Data Creation
import openai
openai.api_key ="STUB"

def get_answer_from_open_ai(a_question):
    messages=[
    {"role": "system", "content": "I am going to give you a question. Please reply with a single sentence that could be an answer to that question"},
    {"role": "user", "content": a_question}
  ]
    response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    return response.choices[0].message.content

In [42]:
subframes[0]['answers'] = subframes[0]['question'].apply(get_answer_from_open_ai)

C:\Users\Cam\AppData\Local\Temp\ipykernel_30852\494842232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subframes[0]['answers'] = subframes[0]['question'].apply(get_answer_from_open_ai)


In [43]:
subframes[0]['answers']

0      The Trump Ocean Club International Hotel and T...
1           Sasha Vujačić was born in Maribor, Slovenia.
2                   Dead Combo was released in Portugal.
3                                        "Marion Bridge"
4                                     The United States.
                             ...                        
495         The film "Paranoia 10" is filmed in English.
496    Wesley van der Linden was born in Amsterdam, N...
497              Dominic Deutscher comes from Australia.
498    Jingmen City-Located on the banks of the Yangt...
499        Jason Macy was born in Auckland, New Zealand.
Name: answers, Length: 500, dtype: object

In [44]:
subframes[0].to_csv('generated_data/it_1.csv', index=False)

In [47]:
from time import sleep

for i in range(2, 8):
    sleep(60)
    subframes[i]['answers'] = subframes[i]['question'].apply(get_answer_from_open_ai)
    subframes[i].to_csv('generated_data/it_' + str(i) + '.csv')

C:\Users\Cam\AppData\Local\Temp\ipykernel_30852\4193552803.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subframes[i]['answers'] = subframes[i]['question'].apply(get_answer_from_open_ai)
C:\Users\Cam\AppData\Local\Temp\ipykernel_30852\4193552803.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subframes[i]['answers'] = subframes[i]['question'].apply(get_answer_from_open_ai)
C:\Users\Cam\AppData\Local\Temp\ipykernel_30852\4193552803.py:5: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [48]:
for i in range(0, 8):
    subframes[i]['answers'] = subframes[i]['answers'].apply(remove_punc)

C:\Users\Cam\AppData\Local\Temp\ipykernel_30852\2147219349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subframes[i]['answers'] = subframes[i]['answers'].apply(remove_punc)


In [49]:
for i in range(0, 8):
    subframes[i].to_csv('generated_and_cleaned/it_' + str(i) + '.csv')